In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from autogen import AssistantAgent, UserProxyAgent

In [3]:
config_list = [{'model': 'deepseek-r1-distill-qwen-32b', 'api_key': os.getenv("GROQ_API_KEY"), 'api_type': "groq"}]

In [4]:
llm_config={"config_list": config_list}

In [6]:
task = '''
 **Task**: As an architect, you are required to design a solution for the
 following business requirements:
    - Data storage for massive amounts of IoT data
    - Real-time data analytics and machine learning pipeline
    - Scalability
    - Cost Optimization
    - Region pairs in Europe, for disaster recovery
    - Tools for monitoring and observability
    - Timeline: 6 months

    Break down the problem using a Chain-of-Thought approach. Ensure that your
    solution architecture is following best practices.
    '''

In [7]:
cloud_prompt = '''
**Role**: You are an expert cloud architect. You need to develop architecture proposals
using either cloud-specific PaaS services, or cloud-agnostic ones.
The final proposal should consider all 3 main cloud providers: Azure, AWS and GCP, and provide
a data architecture for each. At the end, briefly state the advantages of cloud over on-premises
architectures, and summarize your solutions for each cloud provider using a table for clarity.
'''
cloud_prompt += task

In [8]:
cloud_prompt

'\n**Role**: You are an expert cloud architect. You need to develop architecture proposals\nusing either cloud-specific PaaS services, or cloud-agnostic ones.\nThe final proposal should consider all 3 main cloud providers: Azure, AWS and GCP, and provide\na data architecture for each. At the end, briefly state the advantages of cloud over on-premises\narchitectures, and summarize your solutions for each cloud provider using a table for clarity.\n\n **Task**: As an architect, you are required to design a solution for the\n following business requirements:\n    - Data storage for massive amounts of IoT data\n    - Real-time data analytics and machine learning pipeline\n    - Scalability\n    - Cost Optimization\n    - Region pairs in Europe, for disaster recovery\n    - Tools for monitoring and observability\n    - Timeline: 6 months\n\n    Break down the problem using a Chain-of-Thought approach. Ensure that your\n    solution architecture is following best practices.\n    '

In [10]:
oss_prompt = '''
**Role**: You are an expert on-premises, open-source software architect. You need
to develop architecture proposals without considering cloud solutions.
 Only use open-source frameworks that are popular and have lots of active contributors.
 At the end, briefly state the advantages of open-source adoption, and summarize your
 solutions using a table for clarity.
'''
oss_prompt += task

In [11]:
lead_prompt =  '''
**Role**: You are a lead Architect tasked with managing a conversation between
the cloud and the open-source Architects.
Each Architect will perform a task and respond with their resuls. You will critically
review those and also ask for, or point to, the disadvantages of their solutions.
You will review each result, and choose the best solution in accordance with the business
requirements and architecture best practices. You will use any number of summary tables to
communicate your decision.
'''
lead_prompt += task

In [13]:
user_proxy=UserProxyAgent(
    name="supervisor",
    system_message = "A Human Head of Architecture",
    code_execution_config={
        "last_n_messages": 2,
        "work_dir": "groupchat",
        "use_docker": False,
    },
    human_input_mode="NEVER",
)

In [14]:
cloud_agent = AssistantAgent(
    name = "cloud",
    system_message = cloud_prompt,
    llm_config={"config_list": config_list}
    )

oss_agent = AssistantAgent(
    name = "oss",
    system_message = oss_prompt,
    llm_config={"config_list": config_list}
    )

lead_agent = AssistantAgent(
    name = "lead",
    system_message = lead_prompt,
    llm_config={"config_list": config_list}
)

#### Communication flow
supervisor_agent-> cloud_agent -> oss_agent -> lead_agent

In [15]:
def state_transition(last_speaker, groupchat):
    messages = groupchat.messages
    
    if last_speaker is user_proxy:
        return cloud_agent
    elif last_speaker is cloud_agent:
        return oss_agent
    elif last_speaker is oss_agent:
        return lead_agent
    elif last_speaker is lead_agent:
        # lead -> end
        return None

In [18]:
from autogen import GroupChat, GroupChatManager

In [19]:
groupchat = GroupChat(
    agents=[user_proxy, oss_agent, lead_agent, cloud_agent],
    messages=[],
    max_round=6,
    speaker_selection_method=state_transition
)

In [20]:
manager = GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [21]:
user_proxy.initiate_chat(
    manager, message="Provide your best architecture based on the AI agent for the business requirements."
)

supervisor (to chat_manager):

Provide your best architecture based on the AI agent for the business requirements.

--------------------------------------------------------------------------------

Next speaker: cloud

cloud (to chat_manager):

<think>
Okay, I need to design a cloud architecture based on the user's business requirements. Let's see, the requirements are data storage for massive IoT data, real-time analytics and machine learning, scalability, cost optimization, disaster recovery in Europe, monitoring tools, and a 6-month timeline. Hmm.

First, I'll break down each requirement. For data storage, since it's IoT data, it's probably high volume and high velocity. So I need a scalable storage solution that can handle a lot of data quickly. Maybe something like Azure IoT Hub, AWS IoT Core, or Google Cloud IoT Core. These services are designed for IoT devices and can handle the ingestion of data at scale.

Next, real-time analytics and machine learning. This means I need a pipe

c:\Users\s.kumar\Documents\python\ss\AutoGen-MultiAgent-System\venv\Lib\site-packages\autogen\oai\groq.py:282: UserWarning: Cost calculation not available for model deepseek-r1-distill-qwen-32b
  warnings.warn(f"Cost calculation not available for model {model}", UserWarning)


oss (to chat_manager):

<think>
Alright, let's break this down. The user has provided a detailed question about designing an architecture for specific business requirements using on-premises, open-source solutions. They want me to act as an expert architect, ignoring cloud solutions and using popular open-source frameworks.

First, I need to understand the business requirements thoroughly. They include massive IoT data storage, real-time analytics and machine learning, scalability, cost optimization, disaster recovery in Europe, monitoring tools, and a 6-month timeline.

I should start by identifying the key components needed for each requirement. For data storage, considering the massive scale of IoT data, a time-series database would be essential. InfluxDB comes to mind since it's popular and open-source. For handling the ingestion from millions of devices, Kafka is a strong candidate as it's widely used for real-time data streaming.

Real-time analytics can be handled with Flink, wh

ChatResult(chat_id=None, chat_history=[{'content': 'Provide your best architecture based on the AI agent for the business requirements.', 'role': 'assistant', 'name': 'supervisor'}, {'content': "<think>\nOkay, I need to design a cloud architecture based on the user's business requirements. Let's see, the requirements are data storage for massive IoT data, real-time analytics and machine learning, scalability, cost optimization, disaster recovery in Europe, monitoring tools, and a 6-month timeline. Hmm.\n\nFirst, I'll break down each requirement. For data storage, since it's IoT data, it's probably high volume and high velocity. So I need a scalable storage solution that can handle a lot of data quickly. Maybe something like Azure IoT Hub, AWS IoT Core, or Google Cloud IoT Core. These services are designed for IoT devices and can handle the ingestion of data at scale.\n\nNext, real-time analytics and machine learning. This means I need a pipeline that can process data as it comes in and

In [22]:
response = user_proxy.initiate_chat(
    manager, message="i have a business problem with education industry we are not able to get proper assistant can you create one sclable chatbot architecture."
)

supervisor (to chat_manager):

i have a business problem with education industry we are not able to get proper assistant can you create one sclable chatbot architecture.

--------------------------------------------------------------------------------

Next speaker: cloud

cloud (to chat_manager):

<think>
Okay, so I need to design a scalable chatbot architecture for the education industry. The user mentioned they're not getting proper assistance, so I guess the main goals are to handle a large number of users efficiently, provide personalized support, and ensure reliability and cost-effectiveness.

First, I should break down the components needed. Starting with the frontend, the chatbot needs to be accessible through various platforms—maybe websites, mobile apps, or even messaging platforms like Slack or Microsoft Teams. It should be user-friendly and responsive.

Next, the backend. The chatbot will need natural language processing (NLP) capabilities to understand and respond to user 

c:\Users\s.kumar\Documents\python\ss\AutoGen-MultiAgent-System\venv\Lib\site-packages\autogen\oai\groq.py:282: UserWarning: Cost calculation not available for model deepseek-r1-distill-qwen-32b
  warnings.warn(f"Cost calculation not available for model {model}", UserWarning)


oss (to chat_manager):

<think>
Okay, I need to design a scalable chatbot architecture specifically for the education industry. The user is facing issues with not getting proper assistance, so the goal is to create a system that can handle a large number of users efficiently, provide personalized support, and ensure reliability and cost-effectiveness.

First, I should identify the key components that need to be addressed. Let me start by breaking down the problem into smaller parts.

**1. User Interface**
The chatbot needs a user-friendly interface where students and educators can interact easily. It should be accessible through various platforms like websites, mobile apps, and even social media or messaging apps. I need to ensure the interface is responsive and can handle a high volume of users simultaneously without lag.

**2. Natural Language Processing (NLP)**
The core of the chatbot is the NLP component. It should be able to understand and respond to queries accurately. Considerin

In [25]:
print(response.chat_history)

[{'content': 'i have a business problem with education industry we are not able to get proper assistant can you create one sclable chatbot architecture.', 'role': 'assistant', 'name': 'supervisor'}, {'content': "<think>\nOkay, so I need to design a scalable chatbot architecture for the education industry. The user mentioned they're not getting proper assistance, so I guess the main goals are to handle a large number of users efficiently, provide personalized support, and ensure reliability and cost-effectiveness.\n\nFirst, I should break down the components needed. Starting with the frontend, the chatbot needs to be accessible through various platforms—maybe websites, mobile apps, or even messaging platforms like Slack or Microsoft Teams. It should be user-friendly and responsive.\n\nNext, the backend. The chatbot will need natural language processing (NLP) capabilities to understand and respond to user queries. Using existing NLP services could save time, but custom models might be ne

In [26]:
print("\n=== Chat Conversation ===")
for i, message in enumerate(response.chat_history):
    print(f"\n**Message {i + 1}**")
    print(f"Role: {message['role']}")  # Role of the sender (e.g., user, assistant)
    print(f"Content: {message['content']}")  # Content of the message
    print("-" * 40)  # Separator for readability


=== Chat Conversation ===

**Message 1**
Role: assistant
Content: i have a business problem with education industry we are not able to get proper assistant can you create one sclable chatbot architecture.
----------------------------------------

**Message 2**
Role: user
Content: <think>
Okay, so I need to design a scalable chatbot architecture for the education industry. The user mentioned they're not getting proper assistance, so I guess the main goals are to handle a large number of users efficiently, provide personalized support, and ensure reliability and cost-effectiveness.

First, I should break down the components needed. Starting with the frontend, the chatbot needs to be accessible through various platforms—maybe websites, mobile apps, or even messaging platforms like Slack or Microsoft Teams. It should be user-friendly and responsive.

Next, the backend. The chatbot will need natural language processing (NLP) capabilities to understand and respond to user queries. Using exi